## Capital Asset Pricing Model 

The source code below is made available for educational purposes only and is not intended to provide valuation advice or recommendation. 

In [1]:
#Import Libraries 
import investpy
from datetime import datetime, timedelta 
from pandas_datareader import data
import pandas_datareader.data as reader
import statsmodels.api as sm 
from scipy import stats
import pandas as pd
import numpy as np

c:\Users\roger\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Determine the stocks

In [2]:
a = "1155.KL"
b = "^KLSE;1=9"

Determine the periods 

In [3]:
#setting today date as End Date
today = datetime.today()
previous = today - timedelta(days=1)
end = today.strftime('%d/%m/%Y')
previous = previous.strftime('%d/%m/%Y')

# Define a start date
start = today.replace(year=today.year-50)
start = start.strftime('%d/%m/%Y')

Risk-Free Rate

In [4]:
Rf = investpy.get_bond_historical_data(bond='Malaysia 10Y', from_date= previous, to_date=end)
Rf.tail()

,Open,High,Low,Close
Date,,,,
2022-06-23,4.274,4.274,4.196,4.196


In [5]:
Rf = Rf.iloc[0]['Close']
Rf = Rf/100
Rf

0.04196

Market Returns

In [6]:
Rm = data.DataReader(b, 'yahoo', start= start, end= end)
Rm = Rm.resample('Y').last().pct_change()
Rm = Rm.dropna(axis=0)
Rm.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-12-31,-0.053267,-0.049467,-0.048797,-0.059121,-0.410219,-0.059121
2019-12-31,-0.053148,-0.059371,-0.049824,-0.060228,0.257239,-0.060228
2020-12-31,0.020842,0.024201,0.021679,0.024201,0.175230,0.024201
2021-12-31,-0.046665,-0.058628,-0.061590,-0.036676,0.344847,-0.036676
2022-12-31,-0.080420,-0.067913,-0.070312,-0.087067,-1.000000,-0.087067


In [7]:
Rm = Rm["Adj Close"].mean()
Rm

0.05116004609244973

Beta

In [8]:
# Input based on the above 
input = [a,b]
# We are using the "Adj close" price of the securities selected 
df = reader.get_data_yahoo(input, start, end)["Adj Close"]
# Calculate the daily returns 
day_ret = df.resample("D").ffill().pct_change()
# Remove the first row - NaN error 
day_ret = day_ret.dropna(axis=0)
# Index selected above 
ind_pct = day_ret[b]
# Stock selected above 
stck_pct = day_ret[a]
# Define the independence variable
ind_pct_sm = sm.add_constant(ind_pct)
# Define model by using ordinary least square approach 
model = sm.OLS (stck_pct, ind_pct_sm)
results = model.fit()
slope, intercept, r, p, std_err  = stats.linregress(ind_pct,stck_pct)
print("Beta = ", slope)

Beta =  1.0065588121208897


c:\Users\roger\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [9]:
slope

1.0065588121208897

Cost of Equity (in percentage)

In [10]:
Discount_Rate = Rf + slope * ( Rm - Rf)
Discount_Rate = Discount_Rate * 100
Discount_Rate

5.122038746627363